# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,georgetown,5.4112,100.3354,26.96,87,20,1.54,MY,1729463424
1,1,chernyshevsk,52.5243,117.0174,-3.66,87,100,7.51,RU,1729463714
2,2,isafjordur,66.0755,-23.1240,0.05,95,100,6.15,IS,1729463715
3,3,stanley,54.8680,-1.6985,11.96,81,20,10.29,GB,1729463661
4,4,bilibino,68.0546,166.4372,-4.92,91,100,4.01,RU,1729463718


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    alpha = 0.4,
    scale = 0.75,
    color = "City"
)

# Display map pot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit desired criteria. 
ideal_weather_df = city_data_df.loc[(city_data_df["Humidity"]<75) & (city_data_df["Humidity"]>50) & (city_data_df["Max Temp"] < 27) &
(city_data_df["Max Temp"] > 6) & (city_data_df["Wind Speed"] < 6)]

In [5]:
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna(how='any')

In [6]:
# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,yorosso,12.3536,-4.7761,26.49,60,23,2.79,ML,1729463729
15,15,puquio,-14.7000,-74.1333,9.54,56,1,4.50,PE,1729463732
18,18,nemuro,43.3236,145.5750,8.42,64,1,4.42,JP,1729463736
26,26,dzhetygara,52.1908,61.2006,6.71,66,99,3.18,KZ,1729463746
34,34,puerto natales,-51.7236,-72.4875,12.25,62,75,4.63,CL,1729463756


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,yorosso,ML,12.3536,-4.7761,60,
15,puquio,PE,-14.7000,-74.1333,56,
18,nemuro,JP,43.3236,145.5750,64,
26,dzhetygara,KZ,52.1908,61.2006,66,
34,puerto natales,CL,-51.7236,-72.4875,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel, create params dicitonary
radius = 10000
categories = f'accommodation.hotel'
limit = 20

params = {"limit":limit,
          "apiKey":geoapify_key,
          "categories":categories
}


In [9]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
yorosso - nearest hotel: No hotel found
puquio - nearest hotel: Hostal Josuef
nemuro - nearest hotel: お宿エクハシ
dzhetygara - nearest hotel: гостиница "Октябрь"
puerto natales - nearest hotel: Dorotea Patagonia Hostel
west island - nearest hotel: Cocos Village Bungalows
albany - nearest hotel: No hotel found
masterton - nearest hotel: Copthorne Hotel Solway Park
sitka - nearest hotel: Aspen Suites Hotel
dunedin - nearest hotel: Scenic Hotel Dunedin City
port lincoln - nearest hotel: Boston Hotel
bethel - nearest hotel: Hampton Inn Danbury
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
palmer - nearest hotel: No hotel found
bom jesus - nearest hotel: Hotel Familiar
severnyy - nearest hotel: Космос
ferkessedougou - nearest hotel: Ferké Hôtel Iroko
marsa alam - nearest hotel: Blue House Hotel
barhi - nearest hotel: No hotel found
talodi - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
melfort - nearest hotel: No hotel found
saint

,City,Country,Lat,Lng,Humidity,Hotel Name
12,yorosso,ML,12.3536,-4.7761,60,No hotel found
15,puquio,PE,-14.7000,-74.1333,56,Hostal Josuef
18,nemuro,JP,43.3236,145.5750,64,お宿エクハシ
26,dzhetygara,KZ,52.1908,61.2006,66,"гостиница ""Октябрь"""
34,puerto natales,CL,-51.7236,-72.4875,62,Dorotea Patagonia Hostel
...,...,...,...,...,...,...
555,cuatro cienegas de carranza,MX,26.9833,-102.0833,55,No hotel found
556,escanaba,US,45.7453,-87.0646,63,Magnuson Grand Pioneer Inn and Suites
562,roma,US,43.2128,-75.4557,72,Hampton Inn
564,kitimat,CA,54.0524,-128.6534,71,MStar Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    alpha = 0.4,
    scale = 0.75,
    color = "City",
    hover_cols = ["Country","Hotel Name"] # add country + hotel name in hover message
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)